# **Custom Object Detection using TF Lite Model Maker**

#### **Step 1: Install the required packages**
Install TF Lite Model Maker to build the object detection model and Pycocotools for evaluation.

In [3]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

Import the required packages.

In [4]:
import numpy as np
import os
import glob
import PIL.Image

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

#### **Step 2: Prepare the dataset** (Optional and to be done on your local machine)

#### **(i) Convert images to JPEG**
The images in your dataset should be of the format `.jpeg`. Sometimes, it is possible that although the extension of the images in your dataset might be `.jpg` or `.jpeg`, its actual format is something else. For example, `image.jpg` might be a `PNG` image and not `JPEG`.

Use this [script](https://github.com/NSTiwari/Custom-Object-Detection-on-Android-using-TF-Lite/blob/master/convert_images_to_jpeg.py) to programatically convert all the images in your dataset in the `JPEG` format just to be sure else you'll face an error ahead.

**Note:** Do this cleaning and conversion on your local machine before you upload it on Kaggle/Google Drive.

#### **(ii) Check image format**
Verfiy the image format after they're converted into `JPEG`.


In [ ]:
img = PIL.Image.open('/content/cartoon-detection/train/images/cartoon57.jpg')
img.format

#### **Step 3: Download dataset from Kaggle/Google Drive**

Fetch the dataset which you've uploaded on Kaggle. You can also use Google Drive to store your dataset and fetch it by mounting Google Drive on Google Colab.

In [ ]:
# Install Kaggle API
!pip install -q kaggle
!pip install -q kaggle-cli

In [7]:
# only for Google Colab
import os
os.environ['KAGGLE_USERNAME'] = "<your_kaggle_username>" 
os.environ['KAGGLE_KEY'] = "<your_kaggle_key>"

In [ ]:
!kaggle datasets download -d nstiwari/cartoondetection --unzip

#### **Step 4: Choose an object detection model architecture to train your model.**

This tutorial uses the EfficientDet-Lite2 model. EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture. 

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Pixel 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>


In [ ]:
spec = model_spec.get('efficientdet_lite2')

#### **Step 5: Load the dataset.**

Load the train and test dataset by passing the `images_dir`, `annotations_dir` and `labels` of the dataset as the parameters.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc("/content/cartoon-detection/train/images", "/content/cartoon-detection/train/annotations", ['doraemon', 'mrbean', 'scooby', 'mickey', 'mcqueen'])

In [ ]:
validation_data = object_detector.DataLoader.from_pascal_voc('cartoon-detection/test/images', 'cartoon-detection/test/annotations', ['doraemon', 'mrbean', 'scooby', 'mickey', 'mcqueen'])

#### **Step 6: Train the TensorFlow model with the training data.**
Start the model training.


In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, validation_data=validation_data)

#### **Step 7: Evaluate the model with the test data.**

After training the object detection model using the images in the training dataset, evalutate the model on the test data to see how it performs against the data it has never seen before. 

In [ ]:
model.evaluate(validation_data)

#### **Step 8: Export as a TensorFlow Lite model.**

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is full integer quantization.

In [ ]:
model.export(export_dir='.')

#### **Step 9: Evaluate the TensorFlow Lite model.**

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop. 
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('model.tflite', validation_data)

You can download the TensorFlow Lite model file using the left sidebar of Colab. Right-click the `model.tflite` file and choose `Download` to download it to your local computer.

After training the model you can use the [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview) to [integrate the object detector into an Android application](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector).